# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 (+5) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 14.10.24 23:59__   
__Жесткий дедлайн: 17.10.24 23:59__

### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [1]:
%config InlineBackend.figure_format = "retina"
%matplotlib inline

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from collections import defaultdict, Counter

import src.tokenizer as tokenizer
import src.utils as utils

In [3]:
nlines = 3040
nsamples = 30
rows = np.random.choice(range(1, nlines), size=nsamples, replace=False)

dataset = pd.read_csv("biotech_news.tsv", sep="\t", skiprows=lambda x: x not in rows and x != 0)
dataset.head()

,text,labels
0,state bank of southern utah date approved apri...,other
1,comment synopsis cell and gene therapy can go ...,"product launching & presentation, executive st..."
2,for the latest on real estate technology from ...,company description
3,follow summary spire is one of the largest nat...,company description
4,true health new mexico insurance gets new owne...,"m&a, alliance & partnership, service & product..."


## Предобработка лейблов


__Задание 1 (1.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [4]:
# inplace меняем наш датасет, тк он оч тяжёлый

binarizer = utils.Binarizer(dataset=dataset, labels_col="labels")
binarizer.transform(dataset, add_every_label=False)

## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [5]:
texts_train, texts_test, y_train, y_test = train_test_split(
    dataset["text"],
    dataset["binary_labels"],
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

__Задание 2 (1.5 балла)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

In [41]:
from nltk.corpus import stopwords
from collections import defaultdict, Counter

STOP_WORDS = set(stopwords.words("english"))

tokenized_texts = dataset["text"].apply(
    lambda x: [word for word in nltk.word_tokenize(x) if word not in STOP_WORDS]
)

all_words = [word for tokens in tokenized_texts for word in tokens]
word_counts = Counter(all_words)

#######################
doc_freq = defaultdict(int)
for tokens in tokenized_texts:
    unique_tokens = set(tokens)
    for token in unique_tokens:
        doc_freq[token] += 1

#######################
num_docs = len(tokenized_texts)

max_doc_freq = 0.5 * num_docs
min_word_count = 3

vocab = {
    word for word in word_counts
    if word_counts[word] >= min_word_count and doc_freq[word] <= max_doc_freq
}

#######################
word2idx = {word: idx for idx, word in enumerate(sorted(vocab))}
idx2word = {idx: word for word, idx in word2idx.items()}


texts_indices = tokenized_texts.apply(
    lambda tokens: [word2idx[word] for word in tokens if word in vocab]
)

texts_tokens = texts_indices.apply(
    lambda indices: [idx2word[idx] for idx in indices]
)

print(len(vocab))
texts_tokens

1046


0     [state, bank, date, approved, april, 4, 2020, ...
1     [comment, cell, gene, therapy, go, long, way, ...
2     [latest, real, estate, technology, craig, rowe...
3     [follow, summary, spire, one, largest, natural...
4     [true, health, mexico, insurance, gets, santa,...
5     [less, months, based, startup, options, trade,...
6     [9, hours, uk, based, digital, therapy, ieso, ...
7     [way2b, wellness, unique, individual, founder,...
8     [gravy, bisto, first, years, ad, brand, winter...
9     [animal, rights, groups, wisconsin, fall, wolf...
10    [visayas, community, medical, center, vcmc, ce...
11    [pioneer, high, income, trust, target, large, ...
12    [immune, cells, brain, join, together, form, n...
13    [read, article, maine, attorney, general, frey...
14    [free, report, one, key, stock, solid, trend, ...
15    [updated, march, 11, 2022, narasimhan, join, c...
16    [comment, fedo, large, urgent, need, insurance...
17    [published, 2022, 10, 45, financial, fca, 

In [60]:
import pandas as pd


# Предположим, что у вас есть pd.Series с именем `texts_series`
# Который содержит списки слов: texts_series = pd.Series([['привет', 'мир'], ['как', 'дела']])

texts_series = texts_tokens

# Функция для обучения BPE
def learn_bpe(corpus, num_merges):
    # Подсчет частот слов
    word_freqs = Counter()
    for tokens in corpus:
        for word in tokens:
            word_freqs[word] += 1

    # Инициализация: разбиваем слова на символы
    vocab = {}
    for word, freq in word_freqs.items():
        chars = tuple(word) + ('</w>',)
        vocab[chars] = freq

    bpe_merges = []
    for i in range(num_merges):
        # Подсчет частот пар символов
        pairs = defaultdict(int)
        for word, freq in vocab.items():
            for j in range(len(word)-1):
                pairs[(word[j], word[j+1])] += freq

        if not pairs:
            break

        # Находим наиболее частую пару
        best_pair = max(pairs, key=pairs.get)
        bpe_merges.append(best_pair)

        # Обновляем словарь
        vocab_new = {}
        bigram = best_pair
        bigram_str = ''.join(bigram)

        for word, freq in vocab.items():
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) -1 and word[i] == bigram[0] and word[i+1] == bigram[1]:
                    new_word.append(bigram_str)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            vocab_new[tuple(new_word)] = freq

        vocab = vocab_new

    return bpe_merges

# Функция для применения BPE к тексту
def apply_bpe(token, bpe_merges):
    token = tuple(token) + ('</w>',)
    idx = 0
    while idx < len(token)-1:
        pair = (token[idx], token[idx+1])
        if pair in bpe_pairs:
            token = token[:idx] + (''.join(pair),) + token[idx+2:]
            if idx != 0:
                idx -= 1
        else:
            idx += 1
    if token[-1] == '</w>':
        token = token[:-1]
    return token

# Обучаем BPE на корпусе
num_merges = 10000  # Количество слияний, можно настроить
bpe_merges = learn_bpe(texts_series, num_merges)

# Создаем множество для быстрого поиска
bpe_pairs = set(bpe_merges)

# Применяем BPE к текстам
def tokenize_texts(corpus, bpe_merges):
    tokenized_texts = []
    for tokens in corpus:
        tokenized_sentence = []
        for token in tokens:
            bpe_tokens = apply_bpe(token, bpe_pairs)
            tokenized_sentence.extend(bpe_tokens)
        tokenized_texts.append(tokenized_sentence)
    return tokenized_texts

tokenized_texts = tokenize_texts(texts_series, bpe_merges)

# Результат: tokenized_texts содержит списки сабвордов для каждого текста


In [88]:
bpe = tokenizer.BPE(num_merges=1000)
bpe.fit(corpus=texts_series)
print(len(bpe.vocab))

1046


In [83]:
len(new_texts[0])

218

__Задание 3 (2 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +5 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [ ]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс. 

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [ ]:
# your code here

### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\ 
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [ ]:
# your code here

__Задание 6 (1 балл).__ В этом задании у вас есть две опции на выбор: добавить __двунаправленность__ для LSTM _или_ добавить __многослойность__. Можно сделать и то, и другое, но дополнительных баллов за это мы не дадим, только бесконечный респект. Обе модификации реализуются довольно просто (буквально 4 строчки кода, если вы аккуратно реализовали модель) и дают примерно одинаковый прирост в качестве. Сделайте выводы: стоит ли увеличивать размер модели в несколько раз?

In [ ]:
# your code here